In [14]:
import pandas as pd

import nltk
from nltk.corpus import stopwords
import spacy

from collections import Counter
from collections import defaultdict

# Uncomment the following lines if you want to use the freestylo package
'''
from freestylo import TextObject
from freestylo import TextPreprocessor
from freestylo.AlliterationAnnotation import AlliterationAnnotation
from freestylo.EpiphoraAnnotation import EpiphoraAnnotation
from freestylo.PolysyndetonAnnotation import PolysyndetonAnnotation
'''

import os
from tqdm import tqdm
import math
import re

import openai
from dotenv import load_dotenv

from sklearn.metrics import accuracy_score

## Load data and models

In [3]:
nltk.download('stopwords') 

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hadrienstrichard/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
nlp_fr = spacy.load("fr_core_news_sm") 

In [5]:
texts = pd.read_parquet('../Data/excerpts_processed.parquet')

texts.head()

,Author,Title,URL,Excerpt_ID,Excerpt_Text,Cleaned_Text,Tokens
index,,,,,,,
0,Charles Nodier,Smarra ou les démons de la nuit: Songes romant...,https://www.gutenberg.org/ebooks/18083,18083_1,n'est souvent déterminée que par un mot. En ce...,n'est souvent déterminée que par un mot. en ce...,"[n'est, souvent, déterminée, que, par, un, mot..."
1,Charles Nodier,Smarra ou les démons de la nuit: Songes romant...,https://www.gutenberg.org/ebooks/18083,18083_2,Le reste ne me regarde point. J'ai dit de qui ...,le reste ne me regarde point. j'ai dit de qui ...,"[le, reste, ne, me, regarde, point, ., j'ai, d..."
2,Charles Nodier,Smarra ou les démons de la nuit: Songes romant...,https://www.gutenberg.org/ebooks/18083,18083_3,"Les sylphes, tout étourdis du bruit de la veil...","les sylphes, tout étourdis du bruit de la veil...","[les, sylphes, ,, tout, étourdis, du, bruit, d..."
3,Charles Nodier,Smarra ou les démons de la nuit: Songes romant...,https://www.gutenberg.org/ebooks/18083,18083_4,"À peine mes yeux sont fermés, à peine cesse la...","à peine mes yeux sont fermés, à peine cesse la...","[à, peine, mes, yeux, sont, fermés, ,, à, pein..."
4,Charles Nodier,Smarra ou les démons de la nuit: Songes romant...,https://www.gutenberg.org/ebooks/18083,18083_5,"C'est en vain que le jour s'éteindrait, tant q...","c'est en vain que le jour s'éteindrait, tant q...","[c'est, en, vain, que, le, jour, s'éteindrait,..."


In [6]:
texts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14722 entries, 0 to 14754
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Author        14722 non-null  object
 1   Title         14722 non-null  object
 2   URL           14722 non-null  object
 3   Excerpt_ID    14722 non-null  object
 4   Excerpt_Text  14722 non-null  object
 5   Cleaned_Text  14722 non-null  object
 6   Tokens        14722 non-null  object
dtypes: object(7)
memory usage: 920.1+ KB


In [7]:
# Load French function words (common stopwords in French) as our function word list.
french_function_words = set(stopwords.words('french'))
print(f"Number of French function words: {len(french_function_words)}")
# Optionally, inspect a few function words
print("Sample function words:", list(french_function_words)[:10])

Number of French function words: 157
Sample function words: ['nos', 'serions', 'ont', 'eusse', 'sois', 'ou', 'étées', 'fusses', 'mes', 'qu']


In [8]:
# Define a set of punctuation marks we will consider for certain features.
punct_set = {'.', ',', ';', '!', '?', ':', '-', '—', '(', ')', '[', ']', '{', '}', '"', "'", '«', '»'}

## Function word frequencies

In [8]:
# Compute total number of words (excluding punctuation) for each text, to use in normalization
texts['WordCount'] = texts['Tokens'].apply(lambda tokens: sum(1 for t in tokens if t not in punct_set))

# Add one feature column for each function word in the list
for fw in french_function_words:
    col_name = f"fw_{fw}"
    # Normalized frequency: count of fw divided by number of words (avoid division by zero)
    texts[col_name] = texts.apply(
        lambda row: row['Tokens'].count(fw) / row['WordCount'] if row['WordCount'] > 0 else 0.0,
        axis=1
    )

# Example: check a few function word feature columns for the first text
print(texts.loc[0, ['Author'] + [f"fw_{w}" for w in list(french_function_words)[:5]]])
print(texts.info())

/var/folders/xn/v7b7dbbn6xg3hzr5h4395gbc0000gn/T/ipykernel_1071/3428490745.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  texts[col_name] = texts.apply(
/var/folders/xn/v7b7dbbn6xg3hzr5h4395gbc0000gn/T/ipykernel_1071/3428490745.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  texts[col_name] = texts.apply(
/var/folders/xn/v7b7dbbn6xg3hzr5h4395gbc0000gn/T/ipykernel_1071/3428490745.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perf

Author       Charles Nodier
fw_avions               0.0
fw_votre                0.0
fw_étiez                0.0
fw_sont            0.000607
fw_auront               0.0
Name: 0, dtype: object
<class 'pandas.core.frame.DataFrame'>
Index: 14722 entries, 0 to 14754
Columns: 165 entries, Author to fw_du
dtypes: float64(157), int64(1), object(7)
memory usage: 19.1+ MB
None


/var/folders/xn/v7b7dbbn6xg3hzr5h4395gbc0000gn/T/ipykernel_1071/3428490745.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  texts[col_name] = texts.apply(


In [9]:
# Define function word groups
func_word_groups = {
    "gfw_article": {"le", "la", "les", "un", "une", "du", "des"},
    "gfw_preposition": {"à", "de", "en", "dans", "avec", "sans", "sous", "sur", "chez"},
    "gfw_pronoun": {"je", "tu", "il", "elle", "nous", "vous", "ils", "elles", "on", "lui", "leur", "se", "me", "te", "moi", "toi", "qui", "que", "quoi"},
    "gfw_conjunction": {"et", "ou", "mais", "donc", "or", "car", "puisque", "lorsque", "parce", "que", "si"},
    "gfw_auxiliary": {"être", "avoir"}
}

# Compute per-group normalized frequencies
for group_name, word_set in func_word_groups.items():
    texts[group_name] = texts.apply(
        lambda row: sum(row['Tokens'].count(w) for w in word_set) / row['WordCount']
        if row['WordCount'] > 0 else 0.0, axis=1
    )

# Example: check the computed features for the first text
print(texts.loc[0, ['gfw_article', 'gfw_preposition', 'gfw_pronoun', 'gfw_conjunction', 'gfw_auxiliary']])

/var/folders/xn/v7b7dbbn6xg3hzr5h4395gbc0000gn/T/ipykernel_1071/3534742479.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  texts[group_name] = texts.apply(
/var/folders/xn/v7b7dbbn6xg3hzr5h4395gbc0000gn/T/ipykernel_1071/3534742479.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  texts[group_name] = texts.apply(
/var/folders/xn/v7b7dbbn6xg3hzr5h4395gbc0000gn/T/ipykernel_1071/3534742479.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has po

fw_article        0.112325
fw_preposition    0.105647
fw_pronoun        0.078931
fw_conjunction    0.063145
fw_auxiliary      0.001214
Name: 0, dtype: object


/var/folders/xn/v7b7dbbn6xg3hzr5h4395gbc0000gn/T/ipykernel_1071/3534742479.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  texts[group_name] = texts.apply(


In [10]:
print(texts.info())

<class 'pandas.core.frame.DataFrame'>
Index: 14722 entries, 0 to 14754
Columns: 170 entries, Author to fw_auxiliary
dtypes: float64(162), int64(1), object(7)
memory usage: 19.7+ MB
None


## Pos tagging

In [ ]:
# Define POS tags of interest
pos_tags = ['NOUN', 'PROPN', 'VERB', 'AUX', 'ADJ', 'ADV', 'PRON', 'DET', 'ADP', 'CCONJ', 'SCONJ', 'INTJ']

# Initialize POS frequency containers
pos_freqs = defaultdict(list)

for tokens in texts['Tokens']:
    doc = nlp_fr(" ".join(tokens))
    total_words = sum(1 for token in doc if token.pos_ != "PUNCT")
    
    pos_counts = Counter(token.pos_ for token in doc if token.pos_ != "PUNCT")
    pos_counts['NOUN'] += pos_counts.get('PROPN', 0)  # Merge PROPN into NOUN
    for pos in pos_tags:
        freq = pos_counts.get(pos, 0) / total_words if total_words > 0 else 0.0
        pos_freqs[pos].append(freq)

# Assign POS frequency features to the DataFrame
for pos in pos_tags:
    col = 'POS_' + ('NOUN' if pos == 'PROPN' else pos)
    if col not in texts.columns:
        texts[col] = pos_freqs[pos]
        
# Example: view the POS distribution for the first text
print(texts.loc[0, ['Author'] + [f"POS_{w}" for w in list(pos_tags)]])


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x110ce1cd0>>
Traceback (most recent call last):
  File "/Users/hadrienstrichard/Library/Caches/pypoetry/virtualenvs/research-paper-9dnSQuhV-py3.11/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [ ]:
print(texts.info())

## Lexical features

In [ ]:
def compute_lexical_metrics(token_list):
    # Remove punctuation and non-word tokens from the list
    words = []
    for token in token_list:
        if token in punct_set:
            continue  # skip punctuation tokens
        # Remove any punctuation attached to the token (e.g., apostrophes or hyphens) for word length calculation
        cleaned_token = re.sub(r'[\W_]+', '', token, flags=re.UNICODE)  # keep only alphanumeric characters
        if cleaned_token == "" or cleaned_token.isdigit():
            continue  # skip if token is empty or purely numeric after cleaning
        words.append(cleaned_token)
    if len(words) == 0:
        # If no valid word tokens, return zeros
        return pd.Series({"lex_TTR": 0.0, "lex_Hapax_Ratio": 0.0, "lex_AvgWordLen": 0.0})
    total_words = len(words)
    unique_words = set(words)
    # Type-Token Ratio
    ttr = len(unique_words) / total_words
    # Hapax Legomena Ratio
    word_freqs = Counter(words)
    hapax_count = sum(1 for count in word_freqs.values() if count == 1)
    hapax_ratio = hapax_count / total_words
    # Average word length (in characters)
    total_chars = sum(len(w) for w in words)
    avg_word_len = total_chars / total_words
    return pd.Series({"lex_TTR": ttr, "lex_Hapax_Ratio": hapax_ratio, "lex_AvgWordLen": avg_word_len})

# Apply the lexical metrics function to each text
lexical_df = texts['Tokens'].apply(compute_lexical_metrics)
# Merge the resulting metrics columns into the main DataFrame
texts = pd.concat([texts, lexical_df], axis=1)

# Example: show lexical richness metrics for a text
print(texts.loc[0, ['Author', 'WordCount', 'lex_TTR', 'lex_Hapax_Ratio', 'lex_AvgWordLen']])
print(texts.info())


Author         Charles Nodier
WordCount                1647
TTR                  0.448087
Hapax_Ratio          0.327869
AvgWordLen           4.870674
Name: 0, dtype: object


## Punctuation

In [ ]:
# Define a mapping of punctuation symbols to descriptive column names
punct_marks = {'.': 'Period', ',': 'Comma', ';': 'Semicolon', '?': 'QuestionMark', '!': 'ExclamationMark', ':': 'Colon'}

for symbol, name in punct_marks.items():
    texts[f"{name}_Count"] = texts['Tokens'].apply(lambda tokens: tokens.count(symbol))

# Example: show punctuation counts for the first text
print(texts.loc[0, ['Author'] + [f"pun_{name}_Count" for name in punct_marks.values()]])


Author                   Charles Nodier
Period_Count                         51
Comma_Count                         116
Semicolon_Count                       6
QuestionMark_Count                    0
ExclamationMark_Count                 2
Colon_Count                           3
Name: 0, dtype: object


In [ ]:
print(texts.info())

## Sentence metrics

In [ ]:
def compute_sentence_metrics_with_complexity(token_list):
    # Count sentence-ending punctuation marks
    sentence_enders = {'.', '!', '?'}
    
    # Count words (exclude all punctuation tokens)
    sent_count = sum(token_list.count(sym) for sym in sentence_enders)
    sent_count = max(sent_count, 1 if any(t not in punct_set for t in token_list) else 0)
    word_count = sum(1 for t in token_list if t not in punct_set)
    avg_sent_len = word_count / sent_count if sent_count > 0 else 0.0

    # Re-parse to extract dependency info
    doc = nlp_fr(" ".join(token_list))
    sub_clause_deps = {"ccomp", "xcomp", "advcl", "relcl", "acl", "mark"}
    sub_clause_count = sum(1 for token in doc if token.dep_ in sub_clause_deps)

    complexity_score = sub_clause_count / sent_count if sent_count > 0 else 0.0
    return pd.Series({
        "Sentence_Count": sent_count,
        "Sentence_AvgLength": avg_sent_len,
        "Sentence_Complexity": complexity_score
    })

sentence_df = texts['Tokens'].apply(compute_sentence_metrics_with_complexity)
texts = pd.concat([texts, sentence_df], axis=1)

In [ ]:
# Define tense/mood patterns to track
tense_patterns = {
    "tense_pres_ind": ("Tense=Pres", "Mood=Ind"),
    "tense_past_ind": ("Tense=Past", "Mood=Ind"),
    "tense_imp_ind": ("Tense=Imp", "Mood=Ind"),
    "tense_cond": ("Mood=Cnd",),
    "tense_subj": ("Mood=Sub",),
    "tense_inf": ("VerbForm=Inf",)
}

# Initialize dict of lists
tense_counts = {k: [] for k in tense_patterns.keys()}

for tokens in texts['Tokens']:
    doc = nlp_fr(" ".join(tokens))
    verb_tokens = [t for t in doc if t.pos_ in {"VERB", "AUX"}]
    total_verbs = len(verb_tokens)
    
    for label, conditions in tense_patterns.items():
        count = sum(
            all(cond in t.morph.to_json() for cond in conditions)
            for t in verb_tokens
        )
        tense_counts[label].append(count / total_verbs if total_verbs > 0 else 0.0)

# Add to DataFrame
for label, vals in tense_counts.items():
    texts[label] = vals


In [ ]:
print(texts.info())

## Character Metrics

In [ ]:
# Commented out as it added too much features for too little gain

'''

def reconstruct_text(token_list):
    """Reconstruct text string from tokens, inserting spaces appropriately."""
    text = ""
    for token in token_list:
        if token in punct_set:
            # Attach punctuation directly to the text (no preceding space)
            text += token
        else:
            # If not the first token, add a space before adding the word
            if text:
                text += " "
            text += token
    return text

# List to collect n-gram frequency dictionaries for each text
char_ngram_features = []

for token_list in texts['Tokens']:
    text = reconstruct_text(token_list)
    # Initialize a dict for this text's n-grams
    ngram_counts = {}
    # Character bigrams (2-grams)
    for i in range(len(text) - 1):
        bigram = text[i:i+2]
        # Use an underscore to represent spaces in feature names for clarity
        feat_name = "bi_" + bigram.replace(" ", "_")
        ngram_counts[feat_name] = ngram_counts.get(feat_name, 0) + 1
    # Character trigrams (3-grams)
    for i in range(len(text) - 2):
        trigram = text[i:i+3]
        feat_name = "tri_" + trigram.replace(" ", "_")
        ngram_counts[feat_name] = ngram_counts.get(feat_name, 0) + 1
    char_ngram_features.append(ngram_counts)

# Convert list of dicts to DataFrame (each n-gram becomes a column, fill missing with 0)
char_ngram_df = pd.DataFrame(char_ngram_features).fillna(0).astype(int)
# Merge the n-gram features into the main DataFrame
texts = pd.concat([texts, char_ngram_df], axis=1)

# Example: number of character n-gram feature columns
print(f"Total character n-gram features: {char_ngram_df.shape[1]}")
# Show a small sample of character n-gram features for the first text
print(texts.loc[0, [col for col in texts.columns if col.startswith('bi_')][:5]])



'''



Total character n-gram features: 28205
bi_n'     20.0
bi_'e     37.0
bi_es    206.0
bi_st     32.0
bi_t_    201.0
Name: 0, dtype: object


## Figure of Speech Tagging

Avec freestylo

In [ ]:
#Commented out as it made the computer freeze

'''

FIGURES = [AlliterationAnnotation,
           EpiphoraAnnotation, PolysyndetonAnnotation]

def extract_stylistic_features(text, lang='fr', nlp=None):
    tobj = TextObject.TextObject(text=text, language=lang)
    tp = TextPreprocessor.TextPreprocessor(language=lang)
    tp.nlp = nlp
    tp.process_text(tobj)

    features, examples = {}, {}
    for AnnotClass in FIGURES:
        annot = AnnotClass(text=tobj)
        annot.find_candidates()
        if hasattr(annot, 'load_model'):
            annot.load_model()
        if hasattr(annot, 'score_candidates'):
            annot.score_candidates()

        instances = annot.candidates 
        features[f"fig_{AnnotClass.__name__}"] = len(instances)
        examples[AnnotClass.__name__] = instances
    return features, examples


def apply_stylistic_features(df, column='Cleaned_Text', nlp=None):
    feats_list, ex_list = [], []

    for txt in tqdm(df[column].astype(str), desc="Extracting stylistic features"):
        feats, ex = extract_stylistic_features(txt, nlp=nlp)
        feats_list.append(feats)
        ex_list.append(ex)

    feats_df = pd.DataFrame(feats_list).fillna(0).astype(int)
    return pd.concat([df.reset_index(drop=True), feats_df], axis=1), ex_list
    
texts, figure_examples = apply_stylistic_features(texts, nlp=nlp_fr)

texts['fig_AlliterationAnnotation', 'fig_EpiphoraAnnotation', 'fig_PolysyndetonAnnotation'].describe()
'''

LLM labeling

In [10]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [34]:
def detect_figures(phrase: str) -> dict:
    prompt = f"""
Tu es un expert en analyse littéraire. Analyse la phrase suivante et indique quelles figures de style elle contient, en te basant sur cette liste. Pour chaque figure, réponds par vrai ou faux. Voici les figures à analyser, avec leur définition et un exemple pour chacune :

1. **Comparaison** : Mise en relation d’un comparé et d’un comparant avec un outil (comme, tel, semblable à, etc.).
   Exemple : « Cet homme est bête comme ses pieds »
2. **Métaphore** : Comparaison sans outil de comparaison.
   Exemple : « Ses cheveux de miel. »
3. **Personnification** : Attribution de caractéristiques humaines à un objet ou un animal.
   Exemple : « Le stylo saute de la table »
4. **Hyperbole** : Exagération dans le but de frapper l’imagination.
   Exemple : « Je meurs de faim »
5. **Litote** : Dire moins pour faire entendre plus.
   Exemple : « Je ne te hais point »
6. **Antithèse** : Opposition très forte entre deux idées.
   Exemple : « Je vis, je meurs »
7. **Oxymore** : Réunion de deux termes opposés dans un même groupe de mots.
   Exemple : « Une obscure clarté »
8. **Répétition** : Reprise d’un même mot plusieurs fois.
   Exemple : « La guerre, la guerre, la guerre ! »
9. **Énumération** : Accumulation de mots de même nature grammaticale.
   Exemple : « Il mange des pommes, des poires, des bananes et des kiwis. »
10. **Euphémisme** : Expression atténuée d’une idée jugée brutale.
   Exemple : « Il nous a quittés » (pour "il est mort")

Phrase à analyser :
« {phrase} »

Réponds uniquement avec un dictionnaire JSON contenant ces clés : "Comparaison", "Métaphore", "Personnification", "Hyperbole", "Litote", "Antithèse", "Oxymore", "Répétition", "Énumération", "Euphémisme", et pour chaque clé, la valeur `true` ou `false` (sans guillemets).

Ne donne pas d’explication.
    """

    response = openai.chat.completions.create(
        model="gpt-4.1-nano-2025-04-14",
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
        max_tokens=512
    )

    # Extraction de la sortie générée
    generated = response.choices[0].message.content

    # Évaluation sécurisée du JSON renvoyé (sans `eval`)
    import json
    try:
        result = json.loads(generated)
    except json.JSONDecodeError:
        print("Parsing error:", generated)
        result = {}

    return result

In [35]:
detect_figures('Je ne te hais point')  # Example usage

{'Comparaison': False,
 'Métaphore': False,
 'Personnification': False,
 'Hyperbole': False,
 'Litote': True,
 'Antithèse': False,
 'Oxymore': False,
 'Répétition': False,
 'Énumération': False,
 'Euphémisme': False}

In [ ]:
# Test the LLM labeling function

# Load the test dataset
fig_df = pd.read_csv(
    "../Data/figures.csv",
    encoding="utf-8",
    sep=";",
)

# Normalize figure labels
fig_df["Figure"] = fig_df["Figure"].str.strip().str.capitalize()

# Store results
predicted_labels = []
correct_flags = []

# Loop through each sentence and evaluate
for i, row in tqdm(fig_df.iterrows(), total=len(fig_df), desc="Running LLM predictions"):
    sentence = row["Texte"]
    true_label = row["Figure"]

    result = detect_figures(sentence)

    # Store full prediction dictionary (optional)
    predicted_labels.append(result)

    # Check if the true label is among the ones flagged as True
    detected_true = result.get(true_label, False)
    correct_flags.append(detected_true)

# Add evaluation columns
fig_df["LLM_Output"] = predicted_labels
fig_df["Correct"] = correct_flags

# Compute accuracy
accuracy = fig_df["Correct"].mean()
print(f"\nAccuracy (relaxed): {accuracy:.2%}")

# Show errors (where the true label was not among the detected ones)
errors_df = fig_df[~fig_df["Correct"]]
print(f"\nTotal errors: {len(errors_df)} / {len(fig_df)}")

# Display mismatches
for idx, row in errors_df.iterrows():
    print("\n---")
    print(f"Sentence      : {row['Texte']}")
    print(f"True Label    : {row['Figure']}")
    print(f"LLM Prediction: {[k for k, v in row['LLM_Output'].items() if v]}")


28


KeyError: 'Figure'

## Final DF

In [37]:
texts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14755 entries, 0 to 12596
Columns: 28396 entries, Author to tri_iht
dtypes: float64(28389), object(7)
memory usage: 3.1+ GB


In [39]:
# Taille du dataset
print("Shape:", texts.shape)

# Types de données
print(texts.dtypes.value_counts())

# Nombre total de valeurs manquantes
print("Missing values total:", texts.isna().sum().sum())

print(texts.isna().sum().sum() / (texts.shape[0] * texts.shape[1]) * 100, "% of total values are missing")


Shape: (14755, 28396)
float64    28389
object         7
Name: count, dtype: int64
Missing values total: 937068
0.22365299898339544 % of total values are missing


In [40]:
# Étape 1 : Ne garder que les features + la colonne "Author"
# On supprime tout sauf "Author" et les colonnes de type numérique (features)
cols_to_keep = ['Author'] + [col for col in texts.columns if col not in ['Title', 'URL', 'Excerpt_ID', 'Excerpt_Text', 'Cleaned_Text', 'Tokens'] and col != 'Author']
texts = texts[cols_to_keep]

# Vérification
print("New shape after column filtering:", texts.shape)

# Étape 2 : Remplir les NaN par la moyenne PAR CLASSE (par 'Author')
# Cela suppose que toutes les colonnes sauf 'Author' sont numériques
feature_cols = [col for col in texts.columns if col != 'Author']

# On remplit les NaN par la moyenne des valeurs pour chaque auteur
texts[feature_cols] = texts.groupby('Author')[feature_cols].transform(lambda x: x.fillna(x.mean()))

# Vérification post-traitement
print("Total remaining NaNs:", texts.isna().sum().sum())


New shape after column filtering: (14755, 28390)
Total remaining NaNs: 936870


In [41]:
# Étape 1 : supprimer les colonnes avec uniquement des NaN
texts = texts.dropna(axis=1, how='all')

# Étape 2 : supprimer les colonnes avec uniquement des zéros (sauf colonne 'Author')
feature_cols = [col for col in texts.columns if col != 'Author']
zero_cols = [col for col in feature_cols if (texts[col] == 0).all()]

print(f"Nombre de colonnes entièrement nulles ou égales à zéro : {len(zero_cols)}")

# Drop them
texts.drop(columns=zero_cols, inplace=True)

# Vérification
print("Shape finale du dataset :", texts.shape)


Nombre de colonnes entièrement nulles ou égales à zéro : 0
Shape finale du dataset : (14755, 28390)


In [43]:
texts.to_parquet("../Data/stylo_embedds.parquet", index=False)